### 1. Import các thư viện cần thiết

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score


### 2. Đọc dữ liệu và chuẩn bị dữ liệu

In [ ]:

# Đọc dữ liệu từ file CSV
data = pd.read_csv('train.csv')

# Bỏ cột Id vì không cần thiết cho dự đoán
data = data.drop(columns=['Id'])

# Tách biến mục tiêu (y) và các đặc trưng (X)
X = data.drop(columns=['SalePrice'])
y = data['SalePrice']

# Hiển thị dữ liệu để kiểm tra
data.head()


### 3. Xác định cột số và cột phân loại

In [ ]:

# Chọn cột số và cột phân loại
num_cols = X.select_dtypes(exclude='object').columns
cat_cols = X.select_dtypes(include='object').columns
num_cols, cat_cols


### 4. Xử lý tiền xử lý dữ liệu

In [ ]:

# Tạo pipeline xử lý cho dữ liệu số
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Điền giá trị thiếu bằng trung bình
    ('scaler', StandardScaler())                  # Chuẩn hóa dữ liệu
])

# Tạo pipeline xử lý cho dữ liệu phân loại
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Điền giá trị thiếu bằng giá trị phổ biến nhất
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Mã hóa one-hot cho dữ liệu phân loại
])

# Kết hợp các pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])


### 5. Chia dữ liệu thành tập huấn luyện và kiểm tra

In [ ]:

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 6. Áp dụng log transformation cho biến mục tiêu

In [ ]:

# Áp dụng log transformation để giảm skewness
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)


### 7. Tiền xử lý dữ liệu đặc trưng

In [ ]:

# Áp dụng tiền xử lý
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


### 8. Định nghĩa các mô hình

In [ ]:

# Định nghĩa các mô hình hồi quy
linear_model = LinearRegression()
ridge_model = Ridge(alpha=1.0)
stacking_model = StackingRegressor(
    estimators=[('lr', linear_model), ('ridge', ridge_model)],
    final_estimator=DecisionTreeRegressor(max_depth=5)
)


### 9. Huấn luyện và đánh giá mô hình

In [ ]:

# Tạo từ điển để lưu kết quả
results = {}

# Huấn luyện và đánh giá Linear Regression
linear_model.fit(X_train_processed, y_train_log)
y_pred_linear_log = linear_model.predict(X_test_processed)
results['Linear Regression'] = {
    'MSE': mean_squared_error(y_test_log, y_pred_linear_log),
    'R2': r2_score(y_test_log, y_pred_linear_log)
}

# Huấn luyện và đánh giá Ridge Regression
ridge_model.fit(X_train_processed, y_train_log)
y_pred_ridge_log = ridge_model.predict(X_test_processed)
results['Ridge Regression'] = {
    'MSE': mean_squared_error(y_test_log, y_pred_ridge_log),
    'R2': r2_score(y_test_log, y_pred_ridge_log)
}

# Huấn luyện và đánh giá Stacking Model
stacking_model.fit(X_train_processed, y_train_log)
y_pred_stacking_log = stacking_model.predict(X_test_processed)
results['Stacking Model'] = {
    'MSE': mean_squared_error(y_test_log, y_pred_stacking_log),
    'R2': r2_score(y_test_log, y_pred_stacking_log)
}


### 10. Hiển thị kết quả

In [ ]:

# Chuyển đổi kết quả thành DataFrame và hiển thị
results_df = pd.DataFrame(results).T
results_df


### 11. Huấn luyện MLPRegressor và vẽ biểu đồ hàm mất mát

In [ ]:

# Huấn luyện MLP với lịch sử mất mát
mlp_model = MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42, warm_start=True)
mlp_model.fit(X_train_processed, y_train_log)

# Lấy hàm mất mát sau mỗi lần lặp
loss_curve = mlp_model.loss_curve_

# Vẽ biểu đồ hàm mất mát
plt.figure(figsize=(10, 5))
plt.plot(range(1, len(loss_curve) + 1), loss_curve, label='Loss')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Biểu đồ hàm mất mát của MLP")
plt.legend()
plt.show()


### 12. Biểu đồ so sánh MSE và $R^2$ của các mô hình

In [ ]:

# Thêm MLP vào bảng kết quả và đánh giá
y_pred_mlp_log = mlp_model.predict(X_test_processed)
results['MLP Regressor'] = {
    'MSE': mean_squared_error(y_test_log, y_pred_mlp_log),
    'R2': r2_score(y_test_log, y_pred_mlp_log)
}

# Chuyển đổi kết quả thành DataFrame
results_df = pd.DataFrame(results).T

# Vẽ biểu đồ so sánh MSE
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
results_df['MSE'].plot(kind='bar', color='skyblue')
plt.title("So sánh MSE của các mô hình")
plt.ylabel("MSE")
plt.xticks(rotation=45)

# Vẽ biểu đồ so sánh R2
plt.subplot(1, 2, 2)
results_df['R2'].plot(kind='bar', color='salmon')
plt.title("So sánh $R^2$ của các mô hình")
plt.ylabel("$R^2$")
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()
